## Depression Tracker


### Hidden Markov Model

***Rabindra Nepal***

Here, we implement the hidden Markov model algorithms and use them design an app - Depression Tracker. 

In [1]:
# dependencies
import math
import random 
import numpy as np

Imagine that you are developing a mobile application called “Depression Tracker” to
detect depression tendency in people. This app will be installed in the user’s smartphone
and will try to determine whether the behavior of the user demonstrates a pattern of
depression. A person with depression tendency is assumed to show depressive behavior
more often than an otherwise healthy person. The app will collect data on user’s observed
actions and based on that will discover the user’s depression tendency that is hidden (or
not directly observable). The app is non-intrusive and will collect data about the user’s
actions passively (without requiring intervention).

For designing this app, you have created a simple model that uses a set of behavioral
identifiers or actions for discovering depression tendency. These actions are detectable or
measurable by the smartphone. The actions are: physical movement, passive social
networking (user doesn’t post any text/image, etc.), active social networking (user posts
text/image, etc.), texting (either via social networking apps or cell phone provider’s
texting service), and access apps/websites for psychological help or improvement.
For this assignment, assume that every individual user’s mind can be in one of the two
hidden states: healthy and depressed. The app intends to deduce these hidden states from
five actions: movement, passive social networking, active social networking, texting, and
accessing psychological apps/sites.

You conducted a controlled experiment on a group of users to determine the observation
or emission probabilities of the actions given the two hidden mental states (healthy and
depressed).

The observations derived from the experiment are summerized below.

## Given information

Creating different matrice/states from the given informations:

#### Observable state codes (O): 

| Observable State | State Code | 
| --- | --- |
| Movement | 0 |
| Passive Social Networking | 1 |
| Active Social Networking | 2 |
| Texting | 3 |
| Access Psych Health App/Sites| 4 |

#### Hidden mental states (S):

|Mental States|
| --- |
|healthy|
|depressed|

#### Observation probability table (B): 

B = [$b_j(m)$] , where $b_j(m)$ $\equiv$ P($O_t=v_m | q_t = S_j$)

|   -  | Movement (0) | Passive Social Networking (1) | Active Social Networking (2) | Texting (3) | Access Psych Health App/Sites (4)|
| --- | --- | --- | --- | --- | -- | 
| depressed | 0.05 | 0.35 | 0.20 | 0.20 | 0.20 |
| healthy | 0.50 | 0.10 | 0.30 | 0.10 | 0.0 |

#### Initial state probability ($\Pi$):

$\Pi = [\pi_i]$ where $\pi_i \equiv P(q_1 = S_i)$

$\Pi = [\pi_{depressed} = 0.50, \pi_{healthy} = 0.50]$

#### Transition probability (A):

A = $[a_{ij}]$ where $a_{ij} \equiv$ P($q_{t+1} = S_j | q_t = S_i$) 

P($q_{t+1} = healthy | q_t = healthy$) = 0.80 $\Rightarrow$ P($q_{t+1} = depressed | q_t = healthy$) = 0.20

P($q_{t+1} = depressed | q_t = depressed$) = 0.999 $\Rightarrow$ P($q_{t+1} = healthy | q_t = depressed$) = 0.001


|  [t $\downarrow$]  [(t+1) $\rightarrow$]  | healthy | depressed |
| --- | --- | -- |
|healthy| 0.80 | 0.20 |
|depressed | 0.001 | 0.999 |

## HMM Model

First, let us implement the data structures required to implement the HMM. The data structures are implemented as python class object in order to make it general.

In [2]:
class PI:
    '''inital proabability matrix'''
    def __init__(self, states_dict=None, initial_proba=None):
        if states_dict is not None:
            self.states_dict = states_dict
        else:
            self.states_dict = {'depressed': 0, 'healthy': 1}
        
        if initial_proba is not None:
            self.initial_proba = initial_proba
        else:
            self.initial_proba = np.array([0.5, 0.5]) # default case
    
    @property
    def vector(self):
        return self.initial_proba

In [3]:
class A:
    '''transition probability matrix
       n_states = number of hidden states
       n_observations = number of possible observations
    '''
    def __init__(self, n_states=None, transition_matrix=None):
        if n_states is not None:
            self.n_states = n_states
        else:
            self.n_states = 2
            
        if transition_matrix is not None:
            self.transition_matrix = transition_matrix
        else:
            self.transition_matrix = np.array([[0.80, 0.20], [0.001,  0.999]]) # default case
            
    @property
    def matrix(self):
        '''transition matrix'''
        if self.transition_matrix.shape != (self.n_states, self.n_states):
            raise ValueError('transition matrix should be of size: ', (self.n_states, self.n_states))
        return self.transition_matrix

In [4]:
class B:
    '''emission/observables proabability matrix'''
    def __init__(self, emission_matrix=None):
        if emission_matrix is not None:
            self.emission_matrix = emission_matrix
        else:
            self.emission_matrix = np.array([[0.05, 0.35, 0.20, 0.20, 0.20], [0.50, 0.10, 0.30, 0.10, 0.0]]) # default case
    
    @property
    def matrix(self):
        return self.emission_matrix

In [5]:
class Observables:
    '''returns a observation vector'''
    def __init__(self, n_observations, observables_dict = None, observation_vector=None):
        self.n_observations = n_observations
        
        if observables_dict is not None:
            self.observables = observables_dict
        else:
            self.observables = {'Movement': 0, 'Passive Social Networking': 1, 'Active Social Networking': 2, 
                                'Texting': 3, 'Access Psych Health App/Sites': 4} # default case
            
        if observation_vector is not None:
            assert len(observation_vector) == self.n_observations
            self.observation_vector = observation_vector
            
        # if not passed: create a random observation vector
        else:
            observed_state = np.zeros(self.n_observations).astype('int32')
            obs_values = list(self.observables.values())
            for i in range(self.n_observations):
                observed_state[i] = random.randint(min(obs_values), max(obs_values))
            self.observation_vector = observed_state
        
    @property
    def vector(self, observed_state=None):
        '''return observables vector'''
        return self.observation_vector
        
    
    def obs_encoding(self, obs_vector=None):
        '''returns the state codes of a observation vector'''
        if obs_vector is not None:
            encodings = []
            for obs in obs_vector:
                for key, value in self.observables.items():
                    if obs == value:
                        encodings.append(key)
                        break
            return np.array(encodings)
        
        else:
            obs_vector = self.obs_vector()
            encodings = []
            for obs in obs_vector:
                for key, value in self.observables.items():
                    if obs == value:
                        encodings.append(key)
                        break
            return np.array(encodings)

In [6]:
# example data
# all the defualt cases of above implementations

In [7]:
obs = Observables(n_observations=4)
ObservationVector = obs.vector

b = B()
matrixB = b.matrix

a = A()
matrixA = a.matrix
matrixA

pi = PI()
vectorPI = pi.vector

In [8]:
ObservationVector

array([2, 1, 2, 1])

In [9]:
# observationVector in terms of ecodings
obs.obs_encoding(ObservationVector)

array(['Active Social Networking', 'Passive Social Networking',
       'Active Social Networking', 'Passive Social Networking'],
      dtype='<U25')

In [10]:
b.matrix

array([[0.05, 0.35, 0.2 , 0.2 , 0.2 ],
       [0.5 , 0.1 , 0.3 , 0.1 , 0.  ]])

In [11]:
a.matrix

array([[0.8  , 0.2  ],
       [0.001, 0.999]])

In [12]:
pi.vector

array([0.5, 0.5])

## Filtering

Forward algorithm: calculates probability of observing sequence $O_1, O_2, ....., O_t$ and the final state being in the state $q_t = S_i$ i.e. the forward algorithm parameter is:
\begin{align}
\alpha_t(i) = P(O_1, O_2, ...., O_t, q_t = S_i | \lambda)
\end{align}

the matrix $\alpha$ is calculated recursively using the relation,
\begin{align}
\alpha_{t+1}(j) = \Big[ \sum_i^N \alpha_t(i) a_{ij} \Big] b_j(O_{t+1})
\end{align}
with initial condition: 
\begin{align}
\alpha_1 = \pi_i(i) b_i(O_1),
\end{align}

In [13]:
def forward(PI, A, B, Observables):
    '''returns the alpha matrix using forward algorithm'''
    N = len(Observables)
    S = PI.shape[0]
    
    # initializing the alpha matrix 
    alpha = np.zeros((N, S))
    
    # base case initialization
    for s in range(S):
        alpha[0, s] = PI[s] * B[s, Observables[0]]
    
    # recursive case
    for t in range(1, N):
        # sum over s2 to make matrix
        for s2 in range(S):
            # Recursion Relation:  alpha_i(s2) = [sum_s1 alpha_(i-1)(s1) as1s2] b_s2(i)
            alpha[t, s2] = sum(alpha[t-1, s1] * A[s1, s2] for s1 in range(S)) * B[s2, Observables[t]]
    
    alpha = alpha.transpose()
    assert alpha.shape == (S, N)
    return alpha

In [14]:
alpha = forward(vectorPI, matrixA, matrixB, ObservationVector)
alpha

array([[0.1       , 0.0280525 , 0.0044918 , 0.00126007],
       [0.15      , 0.016985  , 0.00677355, 0.00076651]])

### Normalized filtering

Normalization for each time step is given by, 
\begin{align}
c_t = \frac{1}{\sum_j \alpha_t(j)}
\end{align}
then the normalized $\hat{\alpha}_t(j)$ at each time step $t$ is given by,
\begin{align}
\hat{\alpha}_t(j) = c_t \alpha_t(j)
\end{align}

In [15]:
# helper function to calculate the c_t
def ct_value(alpha_t):
    '''returns the c_t value for each time step t
       takes alpha for time step t (a column vector - as an array)
       and returns the corresponding c_t
    '''
    alpha_sum = 0.0
    for i in range(len(alpha_t)):
        alpha_sum += alpha_t[i]
        
    # checking if alpha_sum
    if alpha_sum == 0.0:
        raise ValueError('sum of alpha values zero!')
    # c_t
    ct = 1.0/alpha_sum
    if ct == 0:
        raise ValueError('c_t is zero - leads to NaN!') 
    return ct

# normalized forward algorithm
def forward_normalized(PI, A, B, Observables):
    '''
        scaled/normalized forward algorithm
            parameters:
                Pi : inital probability of states / array
                A: transition probability matrix
                B: emission probability
                Observables: an array of observables
            returns:
                a scaled alpha matrix: alpha_scaled
                clist: list of c_t for t = 1 to len(Observables)
    '''
    # sizes
    N = len(Observables)
    S = PI.shape[0]
    
    # initialize the base case
    alpha = np.zeros((N, S))
    for s in range(S):
        alpha[0, s] = PI[s] * B[s, Observables[0]]
    
    # scaling parameters, c_t
    clist = list()
    c_1 = ct_value(alpha[0])
    clist.append(c_1)
    
    # initializing the scaled alpha: alpha_scaled
    alpha_scaled = np.zeros((N, S))
    for s in range(S):
        alpha_scaled[0, s] = c_1 * alpha[0, s]
    
    for t in range(1, N):
        for s in range(S):
            alpha[t, s] = sum((alpha_scaled[t-1, i] * A[i, s] * B[s, Observables[t]]) for i in range(S))
            
        # calculating and saving c_t values  
        c_t = ct_value(alpha[t])
        clist.append(c_t)
        
        # updating alpha_scaled matrix
        for s in range(S):
            alpha_scaled[t][s] = c_t * alpha[t][s]
    
    assert len(clist) == len(Observables)
    alpha_scaled = alpha_scaled.transpose()
    return alpha_scaled, np.array(clist)

In [16]:
noramlized_alpha, clist = forward_normalized(vectorPI, matrixA, matrixB, ObservationVector)
print('')
print('normalized matrix alpha: \n', noramlized_alpha)
print('')
print('list of c_t: ', clist)


normalized matrix alpha: 
 [[0.4        0.62286983 0.39872675 0.62177115]
 [0.6        0.37713017 0.60127325 0.37822885]]

list of c_t:  [4.         5.55092978 3.99787792 5.55877753]


## Evaluation

#### Evaluation_unnormalized:
This is simply: $\alpha = \sum_j \alpha_t (j) = [\alpha_1, \alpha_2, ......, \alpha_T]$ i.e. an array of all $\alpha$'s for each time step summed over all the hidden states.

In [17]:
def evaluation_unnormalized(alpha):
    # summing over the hidden states
    return np.sum(alpha, axis=0)

In [18]:
evaluation_unnormalized(alpha)

array([0.25      , 0.0450375 , 0.01126535, 0.00202659])

#### Evaluation_normalized:

$\log P = - \sum_t \log (c_t)$

In [19]:
def evaluation_normalized(clist):
    '''returns the log of probability of a observation sequence
    '''
    # logP = -sum_t(log(c_t))
    logP = -sum([math.log(c) for c in clist])
    # returning the normalized probability
    # return math.exp(logP)
    return logP

In [20]:
evaluation_normalized(clist)

-6.201401718499891

## Backward Probability

Backward variable,
\begin{align}
\beta_t(i) = P(O_{t+1:T} | q_t = S_i, \lambda)
\end{align}
is the probability of being in state $q_t = S_i$ at time $t$ and observing the sequence $O_{t+1}, O_{t+2}, ......, O_{T}$, and can be recursively calculated as, 

\begin{align}
\beta_t (i) = \sum_{j=1}^N b_j (O_{t+1}) \beta_{t+1}(j) a_{ij}
\end{align}

In [21]:
def backward(PI, A, B, Observables):
    '''returns the beta matrix'''
    # sizes
    N = len(Observables)
    S = len(PI)
    
    # initializing beta
    beta = np.zeros((N, S))
    for s in range(S):
        beta[N-1][s] = 1.0
    
    # recursive updates in beta
    for t in reversed(range(N-1)):
        for s in range(S):
            beta[t][s] = sum((A[s, j] * B[j, Observables[t+1]] * beta[t+1, j]) for j in range(S))
            
    beta = beta.transpose()
    assert beta.shape == (S, N)
    return beta

In [22]:
beta = backward(vectorPI, matrixA, matrixB, ObservationVector)
beta

array([[0.0157263 , 0.054015  , 0.3       , 1.        ],
       [0.00302639, 0.03010492, 0.10025   , 1.        ]])

### Normalized backward proability

Normalizing the matrix $\beta$:
\begin{align}
\hat{\beta}_{t}(i) = c_t \beta_t (i), \text{where:} \quad  c_t = \frac{1}{\sum_j \alpha_t(j)}
\end{align}

Here, we use the same normalization constants $c_t$ obtained from the forward algorithm. Therefore,
\begin{align}
\sum_i \hat{\beta_t}(i) \ne 1.
\end{align}

In [23]:
def backward_normalized(PI, A, B, c, Observables):
    '''returns the normalized beta matrix: using ct from forward algorithm'''
    N = len(Observables)
    S = len(PI)
    
    beta = np.zeros((N, S))
    beta_scaled = np.zeros((N, S))
    
    for s in range(S):
        beta[N-1][s] = 1.0
        try:
            beta_scaled[N-1][s] = c[N-1] * 1.0
        except:
            print('Exception in beta_scaled: ', N-1)
            
    for t in reversed(range(N-1)):
        beta_local = {}
        for s in range(S):
            beta_local[s] = sum((A[s, i] * B[i, Observables[t+1]] * beta_scaled[t+1, i]) for i in range(S))
        
        for s in range(S):
            beta_scaled[t][s] = c[t] * beta_local[s]
    
    beta_scaled = beta_scaled.transpose()
    assert beta_scaled.shape == (S, N)
    return beta_scaled

In [24]:
normalized_beta = backward_normalized(vectorPI, matrixA, matrixB, clist, ObservationVector)
normalized_beta

array([[7.7599882 , 6.66329338, 6.66699418, 5.55877753],
       [1.4933412 , 3.71374521, 2.22788722, 5.55877753]])

## Smoothing by Forward-Backward Algorithm


\begin{align}
P(q_t = S_i | O_{1:T}, \lambda) = \frac{P(O_{1:t}, q_t = S_i, \lambda) P(O_{t+1:T} | q_t = S_i, \lambda)}{ \sum_j P(O_{1:T}| q_t = S_j, \lambda) P( q_t = S_j| \lambda)}
\end{align}

\begin{align}
\gamma_t(i) = P(q_t = S_i | O_{1:T}, \lambda)  = \frac{\alpha_t(i) \beta_t(i)}{\sum_j \alpha_t(j) \beta_t(j)}
\end{align}


In [25]:
def smoothed_proability(alpha, beta, noOfStates, noOfTimeSteps):
    '''retunrs the smoothed gamma matrix'''
    
    # initializing the gamma matrix
    alpha = alpha.transpose()
    beta = beta.transpose()
    gamma = np.zeros((noOfTimeSteps, noOfStates))
    
    # first fill gamma with alpha * beta
    for t in range(noOfTimeSteps):
        for s in range(noOfStates):
            gamma[t, s] = alpha[t, s] * beta[t, s]
    
    # print('gamma: ', gamma.transpose())
    # normalizing the gamma matrix
    gamma_norm = np.zeros((noOfTimeSteps, noOfStates))
    for t in range(noOfTimeSteps):
        for s in range(noOfStates):
            
            # normalization: sum in denominator
            gamma_sum = sum(gamma[t][j] for j in range(noOfStates))
            
            # normalzing each element of gamma
            gamma_norm[t, s] = gamma[t, s]/gamma_sum
    
    return gamma_norm.transpose()

In [26]:
smoothed_proability(noramlized_alpha, normalized_beta, len(vectorPI), len(ObservationVector))

array([[0.77599882, 0.74768815, 0.66492999, 0.62177115],
       [0.22400118, 0.25231185, 0.33507001, 0.37822885]])